# Interacting with the Blackboard

In [1]:
import urllib, requests, json, time
bb_url = 'https://translator.ncats.io/blackboard/api'

In [2]:
#query = {"type": "query", "name": "A simple blackboard example","term": "asthma"}
query = {"type":"drug", "uri":"https://pharos.ncats.io/idg/api/v1/ligands(31242)", 
         "name":"imatinib", "description": "This is an example query seeded with gleevec"}
json.dumps(query)
req = requests.post(bb_url, json=query)
req.status_code

200

Having posted the query, we then retrieve the resultant knowledge graph (KG) is JSON form. The blackboard can contain multiple knowledge graphs (one for each query submitted to it). So we identify the KG with the highest id, which should correspond to the query we just submitted

In [3]:
kg = requests.get(bb_url).json()
latest_kg_id = max([x['id'] for x in kg])
kg_url = bb_url+"/"+str(latest_kg_id)
print('Currently have %d KGs. Will use KG ID=%d, %s' % (len(kg), latest_kg_id, kg_url))

Currently have 8 KGs. Will use KG ID=855, https://translator.ncats.io/blackboard/api/855


We then update the KG using the Pharos KS (note this needs to be a `PUT` request)

In [4]:
req = requests.put(kg_url+"/ks.pharos")
req.status_code
time.sleep(10)

Now we retrieve the current state of the KG


In [5]:
req = requests.get(kg_url)
req.json()

{'edgeCount': 6,
 'edges': [{'directed': False,
   'id': 1846,
   'name': None,
   'source': 856,
   'target': 857,
   'type': 'inhibitor'},
  {'directed': False,
   'id': 1847,
   'name': None,
   'source': 856,
   'target': 858,
   'type': 'inhibitor'},
  {'directed': False,
   'id': 1848,
   'name': None,
   'source': 856,
   'target': 859,
   'type': 'inhibitor'}],
 'id': 855,
 'name': 'imatinib',
 'nodeCount': 4,
 'nodes': [{'degree': 3,
   'description': 'This is an example query seeded with gleevec',
   'id': 856,
   'inDegree': 0,
   'name': 'imatinib',
   'outDegree': 3,
   'tags': ['KQuery', 'drug', 'KG:855'],
   'type': 'drug',
   'uri': 'https://pharos.ncats.io/idg/api/v1/ligands(31242)'},
  {'degree': 1,
   'id': 857,
   'inDegree': 1,
   'name': 'Tyrosine-protein kinase ABL1',
   'outDegree': 0,
   'synonyms': ['4J9D',
    '4J9F',
    '5DC0',
    '5DC4',
    '5DC9',
    '1AB2',
    '1ABL',
    '1AWO',
    '1BBZ',
    '1JU5',
    '1OPL',
    '1ZZP',
    '2ABL',
    '2E2B',

As noted in the [documentation](https://spotlite.nih.gov/ncats/blackboard) we can rerun the Pharos KS to add more nodes and edges. The first call, above, added nodes that were related to the query node (and the associated edges). When we call it again, it will add nodes that are related to the nodes added in the previous run (i.e., 2nd neighbors of the query node)

In [6]:
req = requests.put(kg_url+"/ks.pharos")
time.sleep(30)

In [7]:
## Dump out the KG JSON for inspection
req = requests.get(kg_url)
ofile = open('kg.json', 'w') ## for reference
ofile.write(req.text)
ofile.close()

Currently the knowledge graph is stored in a [Neo4J](https://neo4j.com/) instance. We can interact with this assuming you have a Neo4J client running and pointed to `blackboard.db` that was created by the blackboard API.

Alternatively, we can parse the JSON representation of the knowledge graph into an `igraph` object and then visualize it using the Cytoscape [extension](https://github.com/cytoscape/jupyter-cytoscape) for Jupyter notebooks. (This is most easily installed if you're using the Anaconda distribution of Python). 

First we make a function to go from the Knowlegde Graph JSON to an `igraph` object

In [8]:
from igraph import *

def kg2ig(kg):
    if kg['type'] != 'kgraph':
        raise Exception("Must provide a JSON kgraph")

    g = Graph(directed=False)

    nodes = kg['nodes']
    for node in nodes:
        d = {}
        for key in node.keys():
            if key in ['inDegree','outDegree','degree', 'id']: continue
            d[key] = node[key]
        g.add_vertex(**d)

    edges = kg['edges']
    for edge in edges:
        s = list(filter(lambda x: x['id'] == edge['source'], nodes))
        t = list(filter(lambda x: x['id'] == edge['target'], nodes))
        if len(s) == 1 and len(t) == 1:
            g.add_edge(s[0]['name'],t[0]['name'], type=edge['type'])
        
    return g

We then use this to get the object and then display it via the Cytoscape extension

In [9]:
req = requests.get(kg_url)
g = kg2ig(json.loads(req.text))
print(g.summary())

IGRAPH UN-T 39 70 -- 
+ attr: description (v), name (v), synonyms (v), tags (v), type (v), uri (v), type (e)


In [13]:
from cyjs import *
cy = cyjs()
display(cy)

In [11]:
cy.deleteGraph()
cy.addGraph(g)
kkLayout = g.layout("kk")
cy.setPosition(kkLayout)
cy.fit(10)

In [12]:
cy.loadStyleFile('bb-style.js')